# Homework 8

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sy
from func import Func

# Inline plotting
%matplotlib inline

# Make sympy print pretty math expressions
sy.init_printing()

---
## Exercise 8.8


<img src="figures/homework-08/exercise-8.8.png" width="600" />


In [2]:
def create_ex88():
    x1, x2 = sy.symbols('x1, x2')
    f = 3*(x1**2  + x2**2)  +   4*x1*x2  + 5*x1  + 6*x2 + 7
    return Func(f, (x1, x2))
ex88 = create_ex88()
ex88

⎡    2                        2           ⎤
⎣3⋅x₁  + 4⋅x₁⋅x₂ + 5⋅x₁ + 3⋅x₂  + 6⋅x₂ + 7⎦

First, we need to rewrite the function $f$ as the following quadratic form:
$$
f(\mathbf{x}) = \frac{1}{2} \mathbf{x}^T \mathbf{Q} \mathbf{x} - \mathbf{b}^T \mathbf{x}
$$

In [3]:
Q, b = ex88.as_quadratic_form_for_gradient_descent()
Q, b

Ignoring constant term: 1


⎛⎡6  4⎤  ⎡-5⎤⎞
⎜⎢    ⎥, ⎢  ⎥⎟
⎝⎣4  6⎦  ⎣-6⎦⎠

Let us ensure that the return $\mathbf{Q}$ and $\mathbf{b}$ are correct:

In [4]:
x1,x2 = sy.symbols('x1, x2')
x = sy.Matrix([[x1],
               [x2]])
sy.simplify(1/2 * x.T * Q * x - b.T * x)

⎡      2                              2         ⎤
⎣3.0⋅x₁  + 4.0⋅x₁⋅x₂ + 5.0⋅x₁ + 3.0⋅x₂  + 6.0⋅x₂⎦

The only difference with this result and the original function $f$ is the constant term. 

In [5]:
ex88

⎡    2                        2           ⎤
⎣3⋅x₁  + 4⋅x₁⋅x₂ + 5⋅x₁ + 3⋅x₂  + 6⋅x₂ + 7⎦

Next, we find the characteristic polynomial of $\mathbf{Q}$ which is given by $\det(\lambda I - Q)$:

In [6]:
lamda = sy.symbols('lambda')
char_poly = sy.det(lamda * sy.eye(2) - Q)
char_poly

Solving the characteristic polynomial, we get:

In [7]:
sy.nonlinsolve([char_poly], lamda)

From this, we know that $\lambda = 2$ and $\lambda = 10$. This means that eigenvalues of $\mathbf{Q}$ are $2$ and $10$. 

We can use SymPy to find the characteristic polynomial and eigenvalues directly:

In [8]:
Q.charpoly()

In [9]:
Q.eigenvals()

*Side Note: The result is interpreted as follows; the matrix $\mathbf{Q}$ has the eigenvalues 2 and 10 and that the eigenvalues 2 and 10 have algebraic multiplicity 1. Algebraic multiplicity is the number of times that an eigenvalue appears in the characteristic polynomial of a matrix. This is important because there are cases where the same eigenvalue appears multiple tiles in the characteristic polynomial of a matrix.*

In [10]:
sy.Matrix([[5, 0],
           [0, 5]]).eigenvals()

To find the range of values for $\alpha$, we use the Theorem 8.3:


<img src="figures/lecture-08/theorem-8.3.png" width="600" />


Since $\lambda_{max}(\mathbf{Q}) = 10$, the largest range of values of $\alpha$ for which the algorithm is globally convergent is $0 < \alpha < 2/10$

---
## Exercise 8.16


<img src="figures/homework-08/exercise-8.16.png" width="600" />


---
<img src="figures/homework-08/exercise-8.16-a.png" width="600" />


We know that $\lVert \mathbf{x} \rVert^2 = \mathbf{x}^T \mathbf{x}$ where $\mathbf{x} \in \mathbb{R}^n$ because by definition  $\lVert \mathbf{x} \rVert = \sqrt{\mathbf{x}^T \mathbf{x}}$, we have:

\begin{align}
\lVert \mathbf{Ax-b} \rVert^2 
&= (\mathbf{Ax} - \mathbf{b})^T(\mathbf{Ax} - \mathbf{b}) \\
&= (\mathbf{Ax})^T (\mathbf{Ax}) - (\mathbf{Ax})^T \mathbf{b} - \mathbf{b}^T(\mathbf{Ax})  + \mathbf{b}^T \mathbf{b} \\
&= \mathbf{x}^T \mathbf{A}^T (\mathbf{Ax}) - 2 (\mathbf{Ax})^T \mathbf{b}  + \mathbf{b}^T \mathbf{b} \\
&= \mathbf{x}^T (\mathbf{A}^T \mathbf{A})\mathbf{x} - 2 \mathbf{b}^T(\mathbf{Ax})  + \mathbf{b}^T \mathbf{b}
\end{align}

The gradient of the function is:
\begin{align}
f'(\mathbf{x}) = 2 (\mathbf{A}^T \mathbf{A})\mathbf{x} - 2 \mathbf{b}^T \mathbf{A}
\end{align}

The Hessian of the function is:
\begin{align}
f''(\mathbf{x}) = 2 (\mathbf{A}^T \mathbf{A})
\end{align}

Notice that $\mathbf{Q} = 2(\mathbf{A}^T \mathbf{A})$. Why? Consider the quadratic function of the form:

$$
g(\mathbf{x}) = \frac{1}{2} \mathbf{x}^T \mathbf{Q} \mathbf{x} - \mathbf{b}^T \mathbf{x}
$$


If wanted to extract $\mathbf{Q}$ out of this expression, we find the Hessian. First, let us compute the gradient of $g$:
$$
g(\mathbf{x}) = \mathbf{Q} \mathbf{x} - \mathbf{b}
$$

The Hessian of $g$ is:
$$
g(\mathbf{x}) = \mathbf{Q}
$$

Therefore, whenever we need to find the $\mathbf{Q}$ for a quadratic function, we simply compute the Hessian and the result is an expression of $\mathbf{Q}$.

---
Let us verify our result using SymPy:

In [11]:
m, n = sy.symbols('m, n')
A = sy.MatrixSymbol('A', m, n)
b = sy.MatrixSymbol('b', m, 1)
x = sy.MatrixSymbol('x', n, 1)

# Turn it into a Matrix
f = sy.Matrix([ (A*x - b).T * (A*x-b) ])
f

⎡⎛  T    T  T⎞           ⎤
⎣⎝-b  + x ⋅A ⎠⋅(-b + A⋅x)⎦

In [12]:
# Find the gradient of the function
gradient = f.jacobian([x])
gradient

⎡   T           ⎤
⎣2⋅A ⋅(-b + A⋅x)⎦

In [13]:
# Find the Hessian of the function
hessian = gradient.jacobian([x])
hessian

⎡   T  ⎤
⎣2⋅A ⋅A⎦

---
<img src="figures/homework-08/exercise-8.16-b.png" width="600" />


In general, the fixed-step-size algorithm is given by:


<img src="figures/lecture-08/fixed-step-size-gradient-algorithm.png" width="600" />


where $\mathbf{g}^{(k)}$ is gradient of the function i.e., $\nabla f(\mathbf{x}^{(k)})$

The fixed-step-size gradient algorithm for the optimisation problem is therefore:
$$
\mathbf{x}^{(k+1)} = \mathbf{x}^{(k)} - \alpha (  2 (\mathbf{A}^T \mathbf{A})\mathbf{x} - 2 \mathbf{b}^T \mathbf{A}  )
$$

---
<img src="figures/homework-08/exercise-8.16-c.png" width="600" />


To solve this exercise, we need to compute the eigenvalues of $\mathbf{Q}$ and find the largest one. Now, the question is how to find $\mathbf{Q}$ given $\mathbf{A}$. Consider the quadratic function of the form:

$$
g(\mathbf{x}) = \frac{1}{2} \mathbf{x}^T \mathbf{Q} \mathbf{x} - \mathbf{b}^T \mathbf{x}
$$

If wanted to extract $\mathbf{Q}$ out of this expression, we find the Hessian. First, let us compute the gradient of $g$:
$$
g(\mathbf{x}) = \mathbf{Q} \mathbf{x} - \mathbf{b}
$$

The Hessian of $g$ is:
$$
g(\mathbf{x}) = \mathbf{Q}
$$

Therefore, whenever we need to find the $\mathbf{Q}$ for a quadratic function, we simply compute the Hessian and the result is an expression of $\mathbf{Q}$.

From exercise a) we know that Hessian and therefore $\mathbf{Q} = 2(\mathbf{A}^T \mathbf{A})$.

In [14]:
A = sy.Matrix([[1, 0],
               [0, 2]])
Q = 2* (A.T * A)

To determine the largest range of values of $\alpha$ for which the fixed-step-size algorithm is globally convergent, we use Theorem 8.3:


<img src="figures/lecture-08/theorem-8.3.png" width="600" />


We compute the eigenvalues of $\mathbf{Q}$:

In [15]:
Q.eigenvals()

Thus, $0 < \alpha < \frac{2}{8}$ or  $0 < \alpha < \frac{1}{4}$

---
## Exercise 8.21


<img src="figures/homework-08/exercise-8.21.png" width="600" />


---
<img src="figures/homework-08/exercise-8.21-a.png" width="600" />


In [16]:
def create_ex821():
    x1, x2 = sy.symbols('x1, x2')
    f = 1 + 2*x1 + 3*(x1**2 + x2**2) + 4*x1*x2
    return Func(f, (x1, x2))
ex821 = create_ex821()
ex821

⎡    2                        2    ⎤
⎣3⋅x₁  + 4⋅x₁⋅x₂ + 2⋅x₁ + 3⋅x₂  + 1⎦

First, we need to rewrite the function $f$ as the following quadratic form:
$$
f(\mathbf{x}) = \frac{1}{2} \mathbf{x}^T \mathbf{Q} \mathbf{x} - \mathbf{b}^T \mathbf{x}
$$

In [17]:
Q, b = ex821.as_quadratic_form_for_gradient_descent()
(Q, b)

Ignoring constant term: 1


⎛⎡6  4⎤  ⎡-2⎤⎞
⎜⎢    ⎥, ⎢  ⎥⎟
⎝⎣4  6⎦  ⎣0 ⎦⎠

Verify that $\mathbf{Q}$ is correct:

In [18]:
sy.expand(Func.create_quadratic(Q/2, b))

⎡    2                        2⎤
⎣3⋅x₁  + 4⋅x₁⋅x₂ - 2⋅x₁ + 3⋅x₂ ⎦

Looks fine. Next, we determine the largest range of values of $\alpha$ for which the fixed-step-size algorithm is globally convergent. For this, we use Theorem 8.3:


<img src="figures/lecture-08/theorem-8.3.png" width="600" />


We compute the eigenvalues of $\mathbf{Q}$:

In [19]:
Q.eigenvals()

The upper range is $2/10 = 1/5$

In [20]:
ex821.find_step_size_range(Q)

---
<img src="figures/homework-08/exercise-8.21-b.png" width="600" />


In [21]:
def create_ex821b():
    Q = [[3, 3],
         [1, 3]]
    b = [16, 23]
    return Func.create_quadratic(Q, b)
ex821b = create_ex821b()
ex821b

[x₁⋅(3⋅x₁ + x₂) + 16⋅x₁ + x₂⋅(3⋅x₁ + 3⋅x₂) + 23⋅x₂]

First, we need to rewrite the function $f$ as the following quadratic form:
$$
f(\mathbf{x}) = \frac{1}{2} \mathbf{x}^T \mathbf{Q} \mathbf{x} - \mathbf{b}^T \mathbf{x}
$$

In [22]:
Q, b = ex821b.as_quadratic_form_for_gradient_descent()
(Q, b)

⎛⎡6  4⎤  ⎡-16⎤⎞
⎜⎢    ⎥, ⎢   ⎥⎟
⎝⎣4  6⎦  ⎣-23⎦⎠

Let us check that $\mathbf{Q}$ is correct.

In [23]:
sy.expand(ex821b._f[0])

In [24]:
sy.expand(Func.create_quadratic(Q/2, b))

⎡    2                         2        ⎤
⎣3⋅x₁  + 4⋅x₁⋅x₂ - 16⋅x₁ + 3⋅x₂  - 23⋅x₂⎦

Looks fine. Next, we determine the largest range of values of $\alpha$ for which the fixed-step-size algorithm is globally convergent. For this, we use Theorem 8.3:


<img src="figures/lecture-08/theorem-8.3.png" width="600" />


We compute the eigenvalues of $\mathbf{Q}$:

In [25]:
Q.eigenvals()

The upper range is $2/10 = 1/5$

In [26]:
ex821b.find_step_size_range(Q)

---
## Exercise 8.25

<img src="figures/homework-08/exercise-8.25.png" width="600" />


---
## Exercise 8.26

<img src="figures/homework-08/exercise-8.26.png" width="600" />
